<a href="https://colab.research.google.com/github/TheOneTrueGuy/DigitalDayDream/blob/main/1simple_SDXL_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First the necessary installs and dependencies for using diffusers and transformers.

In [ ]:
!pip install --upgrade diffusers transformers accelerate

In [ ]:
#@title You gotta use this too if you want easy save to your google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!wget "https://drive.google.com/file/d/196HuG4w9w_crcIjKaMGMxDme2R9UOS-F/view?usp=drive_link" -O "v2_lora_RollingClockwise.ckpt"

Otherwise you can just download the final zip or .mp4 file.

In [ ]:
#@title imagemagick gives us access to a simple, fast zoom function
!apt install imagemagick
# define zoom function using imagemagick convert (superior pixel and color handling, less pixelation than a simple resize)
def zoom(imagename, zoomscale=1.1):
  os.system(f"convert {imagename} -distort SRT {zoomscale},0 -gravity center {imagename}")



In [ ]:
# run this first then make anim in next cell
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image
#from diffusers import DiffusionPipeline
import datetime as dt
import time
import os
import random
from PIL import ImageFont, ImageDraw, Image
# fakesafety keeps the nsfw filter from returning black squares which will ruin our animations
class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)

# assign simple generative pipe using the base function for use with LoRa layers.
#if not using LoRa, comment this out and switch to next pipe
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)

# this one is better if no LoRa is used, just remove the # from the next 3 lines.
#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16
#)

#experimental
#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "chillpixel/starlight-animated-sdxl", torch_dtype=torch.float16
#)
#chillpixel/starlight-animated-sdxl

# load lora moved to separate cell for optionality

pipe = pipe.to("cuda")
pipe.safety_checker = FakeSafety()



In [ ]:
#@title Load LoRa layer(s) this is optional, don't run if you just want to use default SDXL

#https://replicate.com/jakedahn/sdxl-isometric-geology
# v2_lora_RollingClockwise.ckpt

pipe.load_lora_weights("goofyai/cyborg_style_xl") # use with "cyborg" or "android" keyword
#pipe.load_lora_weights("datamonet/Makima_Chainsaw_Man_LoRA") #needs weight_name
#pipe.load_lora_weights("")

#pipe.load_lora_weights("jakedahn/sdxl-isometric-geology") #spare for swapping in new loras
#pipe.load_lora_weights("") #spare for swapping in new loras

#FFusion/FFusionXL-LoRa-SDXL-Potion-Art-Engine

# cadaeic/duelyst-landscape-lora
# Duskfallcrew/the-crystal-exarch-15
# datamonet/Makima_Chainsaw_Man_LoRA
# the following might work with some param tweaking but I didn't have good luck with these LoRas
# Norod78/sdxl-pumpkin-head-lora #not good
# lora-library/tekakutli-dinosaurs #didn't work

In [ ]:
#@title upload start image 512x512 will be quick and look good

import os

from google.colab import files
uploaded = files.upload()
name=uploaded.keys()
name = list(name)[0]
print(name)


In [ ]:

zip_name = "easy_to_remember_name5" #@param {type:"string"}
negprompt = " monochrome, dumb, flat, boring, ordinary" #@param {type:"string"}
prompt= "Many happy robots dancing in the mechanical jungle of gears, axles and vine like chains stretching from telephone poles and silver pillars" #@param {type:"string"}
zoom_scale=0.99 #@param {type:"number"}
# eventually we will add a math function to the zoom scale and/or keyframe triggers to change the rate.
# notice! zoom scale above 1 like 1.05 will zoom in, below 1, like 0.995 will zoom out
total_frames=300 #@param {type:"number"}

# needs zip_name check to see if file already exists, append random number to prevent overwrite
zip_name=zip_name+".zip"




outdir = dt.datetime.now().strftime("%Y%m%d%H%M")
# if outdir exists add 4 random digits to it:
if os.path.exists(outdir):
    # Generate 4 random digits
    random_digits = str(random.randint(1000, 9999))

    # Create a new output directory with the random digits appended
    new_outdir = outdir + random_digits

    os.mkdir(new_outdir)
    outdir=new_outdir
    print("Created new output directory:", new_outdir)
else:
    os.system(f"mkdir {outdir}")
print("date time now:" + outdir)

xz=0


total=total_frames


# main loop
for xn in range(0,total,1):
  if xn%50==49: torch.cuda.empty_cache()
  init_image = load_image(name).convert("RGB")#

# low step numbers allow the image continuity to not get 'overwritten' by too many fine details in each new generation
  image = pipe(
               prompt,
               num_inference_steps=10,
               strength=0.81,
               guidance_scale=7.34,
               negative_prompt = negprompt ,
               image=init_image
               ).images[0]
  outname= name

  out2=outdir+"/st-" + str(xz).zfill(4) + ".png"
  image.save(out2) # save animation frame
  xz+=1
  image.save(outname)
  # Check if zoom_scale is not equal to zero before calling the zoom function
  if zoom_scale != 0:
    zoom(outname, zoom_scale)

  #next 2 lines apply motion distortion from moto*.txt motion files
  # deleted lines to be replaced soon

# all done, zip the files and save them
os.system(f"zip -r {zip_name} {outdir}")
os.system(f"cp {zip_name}  /content/drive/MyDrive")


In [ ]:
# ffmpeg call to make .mp4 will go here soon

In [ ]:
#run this cell after starting cell(s) above if you don't want to burn
#google compute credits unnecessarily.
from google.colab import runtime
runtime.unassign()